In [1]:
import plotly.express as px
import pandas as pd
import json
from pathlib import Path
import numpy as np
from llama.main import compute_metrics_from_files

In [3]:
path_data = Path("../../data/train_class/metrics.json")
df = pd.read_json(path_data)
df["accuracy"] = df["conf_matrix"].apply(lambda x:np.sum(np.diag(x))/np.sum(x))
df["f1_non_severe"] = df["f1"].apply(lambda x:x[0])
df["f1_severe"] = df["f1"].apply(lambda x:x[1])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 31 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   path_data_json        19 non-null     object 
 1   path_data_folder      19 non-null     object 
 2   algorithm             19 non-null     object 
 3   token                 19 non-null     object 
 4   interval_idx          0 non-null      float64
 5   n_chunks              0 non-null      float64
 6   seed_start            19 non-null     int64  
 7   seed_end              19 non-null     int64  
 8   pred_field            19 non-null     object 
 9   input_field           19 non-null     object 
 10  n_tokens_infered_max  19 non-null     int64  
 11  n_tokens_show_min     19 non-null     int64  
 12  n_tokens_show_max     19 non-null     int64  
 13  n_data                19 non-null     int64  
 14  id                    19 non-null     object 
 15  model_name            19 

In [4]:
fig = px.parallel_coordinates(
    df,
    dimensions=["qlora_alpha", "qlora_dropout", "qlora_r", "lr", "epoch", "f1_severe", "f1_non_severe", "accuracy"],
    color="accuracy",
)
fig.show()

In [5]:
fig.write_image(path_data.parent / "paralell.png")

In [6]:
i = 0
conf_matrix = np.array(df["conf_matrix"].iloc[i])
f1 = df['f1'].iloc[i]

In [7]:
compute_metrics_from_files(
    conf_matrix=conf_matrix,
    f1=f1,
    folder_out=path_data.parent
)

c:\Users\robin\Documents\projets\severityPrediction\src\llm\llama\main.py:649: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown

